# cheerio / axios를 이용해서 웹크롤링을 해보자~
SYNOPSIS<br>
    [.도움 (.help)]<br>
    [.뉴스 (.news)]<br>
<br>
DESCRIPTION<br>
    .help: 도움말<br>
    .news: 최신보안뉴스<br>
<br>
EXTENSION<br>
    [.실검 (.realtime)]<br>
    [.날씨 지역 (.weather local)]<br>
    [.환율 (.exchange)]<br>
    [.운세 (.luck)]<br>
    [.운세 띠 (.luck zodiac_name)]<br>
    [.로또 (.lotto)]<br>
    [.음식메뉴 (.foodmenu)]<br>
    [.소라고동 내용 (.conch text)]<br>
    [.노래추천 (.music)]<br>


## 네이버 뉴스 취득
* 이런식으로 패키지를 불러드릴 수 있다.
* 아래의 라이브러리가 안 깔려 있을 경우에 [npm install axios]와 같이 명령어를 실행해준다.
* 처음 시도할 시에는 [npm init] 명령어로 프로젝트를 작성할 필요가 있음..ㅎㅎ

In [51]:
var axios    = require('axios')
var cheerio  = require('cheerio')
var iconv    = require('iconv-lite');
var log      = console.log

### 아래의 URL의 사이트에서 20대에게 인기있는 10가지 기사를 취득해보자
* URL : https://news.naver.com/main/ranking/popularLike.nhn?subType=20



In [47]:
async function getNews() {
    var url = 'https://news.naver.com/main/ranking/popularLike.nhn?subType=20'
    var NAVER = "https://news.naver.com"
    var str   = ""
    var data  = ""
    try {
        data = await axios.get(url)
        data = data.data
    } catch (err) {
        console.log(err)
    }
    const $ = cheerio.load(data, {decodeEntities: false});
    const $bodyList = $("div.likeitnews_item_text")
    $bodyList.each(function() {
    str += `
${NAVER + $(this).find('a.likeitnews_item_title').attr("href")}
<br>
${$(this).find('a.likeitnews_item_title').html()}
<br>
${$(this).find('div.likeitnews_item_lede').html()}
<br>
<br>
`
    })
    console.log(str)
}
getNews()


https://news.naver.com/main/ranking/read.nhn?rankingType=popular_like&oid=215&aid=0000879313&date=&type=1&rankingSectionId=001&rankingSeq=1
<br>
'��ϱ� ȯ��' �ᱹ �������Ρ�'1,900��' 3�� �߰� �ݿ�
<br>
[�ѱ�����TV ������ ����] ������ '�ڷγ�19' ���¿� ���� ���л����� ��ϱ� ȯ�� �䱸�� ����, 3�� �߰���������(�߰�)�ȿ� ���� ������ �ݿ��ϴ� ��
<br>
<br>

https://news.naver.com/main/ranking/read.nhn?rankingType=popular_like&oid=015&aid=0004362083&date=&type=1&rankingSectionId=001&rankingSeq=2
<br>
"���� �����鼭"������� ���κ��� ���� '���ĸ���'
<br>
ģ�� ���� ����� ����� ��( ���� )�� ����� ���� "���� �����鼭"��� ���ε��� �����ϴ� ���� �߾��� �� ������ �ϰ� �ִ�. �� ���� 17�� ���� TB��
<br>
<br>

https://news.naver.com/main/ranking/read.nhn?rankingType=popular_like&oid=023&aid=0003539716&date=&type=1&rankingSectionId=001&rankingSeq=3
<br>
[�ܵ�] ������� "���� ���� ������, ������ �Ǹ�������"
<br>
�����Ρ��ӵ��� �� ���ε�� û�ʹ� ���� "���ϰ��� ���� �ּ� ���ߴµ�... �׷��� �γ��ϸ� ��� ã�ڴ�" ������ ������� 17�� ������ ���Ͽܱ��Ⱥ���
<br>
<

##### 아니.. 성공했다고 생각했더니 위와 같은 문자깨짐이 일어난다..;

아마도 문제는

euc-kr -> UTF-8로 변환을 하고 있기 때문일 것으로 추측..

axios로 데이터를 취득 시 변환작업을 안하도록 설정을 해보자~

In [48]:
async function getNews() {
    var url = 'https://news.naver.com/main/ranking/popularLike.nhn?subType=20'
    var NAVER = "https://news.naver.com"
    var str   = ""
    var data  = ""
    try {
        data = await axios.get(url)
        data = data.data
    } catch (err) {
        console.log(err)
    }
    const $ = cheerio.load(data, {decodeEntities: false});
    const $bodyList = $("div.likeitnews_item_text")
    $bodyList.each(function() {
    str += `
${NAVER + $(this).find('a.likeitnews_item_title').attr("href")}
<br>
${$(this).find('a.likeitnews_item_title').html()}
<br>
${$(this).find('div.likeitnews_item_lede').html()}
<br>
<br>
`
    })
    console.log(str)
}
getNews()


https://news.naver.com/main/ranking/read.nhn?rankingType=popular_like&oid=215&aid=0000879313&date=&type=1&rankingSectionId=001&rankingSeq=1
<br>
'��ϱ� ȯ��' �ᱹ �������Ρ�'1,900��' 3�� �߰� �ݿ�
<br>
[�ѱ�����TV ������ ����] ������ '�ڷγ�19' ���¿� ���� ���л����� ��ϱ� ȯ�� �䱸�� ����, 3�� �߰���������(�߰�)�ȿ� ���� ������ �ݿ��ϴ� ��
<br>
<br>

https://news.naver.com/main/ranking/read.nhn?rankingType=popular_like&oid=015&aid=0004362083&date=&type=1&rankingSectionId=001&rankingSeq=2
<br>
"���� �����鼭"������� ���κ��� ���� '���ĸ���'
<br>
ģ�� ���� ����� ����� ��( ���� )�� ����� ���� "���� �����鼭"��� ���ε��� �����ϴ� ���� �߾��� �� ������ �ϰ� �ִ�. �� ���� 17�� ���� TB��
<br>
<br>

https://news.naver.com/main/ranking/read.nhn?rankingType=popular_like&oid=023&aid=0003539716&date=&type=1&rankingSectionId=001&rankingSeq=3
<br>
[�ܵ�] ������� "���� ���� ������, ������ �Ǹ�������"
<br>
�����Ρ��ӵ��� �� ���ε�� û�ʹ� ���� "���ϰ��� ���� �ּ� ���ߴµ�... �׷��� �γ��ϸ� ��� ã�ڴ�" ������ ������� 17�� ������ ���Ͽܱ��Ⱥ���
<br>
<